In [36]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip list

Package                   Version
------------------------- --------------
aiohappyeyeballs          2.4.3
aiohttp                   3.10.10
aiosignal                 1.3.1
annotated-types           0.7.0
anyio                     4.6.2.post1
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
colorama                  0.4.6
comm                      0.2.2
dataclasses-json          0.6.7
debugpy                   1.8.7
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.1.0
fastjsonschema            2.20.0
fqdn                      1.5.1
frozenlist                1.4.1
greenlet                  3.1.1
h11       

In [2]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars


import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv



In [3]:
from langchain.chat_models import ChatOpenAI 

In [4]:
pip install json_repair

Note: you may need to restart the kernel to use updated packages.


In [5]:
import json_repair

In [40]:
load_dotenv()

True

In [8]:
graph = Neo4jGraph()

In [9]:
loader = TextLoader(file_path="dummytext.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [10]:
llm_type = os.getenv("LLM_TYPE", "ollama")

# Vérifie si le type de LLM est 'ollama'
if llm_type == "ollama":
    llm = ChatOllama(model="llama3.1", temperature=0)
else:
    # Si ce n'est pas 'ollama', utilise OpenAI GPT-4 à la place
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# Crée un transformateur LLM pour convertir les documents
llm_transformer = LLMGraphTransformer(llm=llm)

# Convertit les documents en graphes avec le transformateur
graph_documents = llm_transformer.convert_to_graph_documents(documents)

C:\Users\franc\AppData\Local\Temp\ipykernel_1268\2963286169.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1", temperature=0)


In [37]:
print(dict(llm))

{'name': None, 'cache': None, 'verbose': False, 'callbacks': None, 'tags': None, 'metadata': None, 'custom_get_token_ids': None, 'base_url': 'http://localhost:11434', 'model': 'llama3.1', 'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': 0.0, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None, 'system': None, 'template': None, 'format': None, 'timeout': None, 'keep_alive': None, 'raw': None, 'headers': None, 'auth': None, 'callback_manager': None, 'rate_limiter': None, 'disable_streaming': False}


In [11]:
graph_documents[0]

GraphDocument(nodes=[Node(id="Amico's Family", type='Family', properties={}), Node(id='The Story of Amico’s Family: A Legacy of Love and Tradition', type='Text', properties={})], relationships=[Relationship(source=Node(id="Amico's Family", type='Family', properties={}), target=Node(id='The Story of Amico’s Family: A Legacy of Love and Tradition', type='Text', properties={}), type='IS_DESCRIBED_IN', properties={})], source=Document(metadata={'source': 'dummytext.txt'}, page_content='1. The Story of Amicoâ€™s Family: A Legacy of Love and Tradition'))

In [12]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [13]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()


GraphWidget(layout=Layout(height='800px', width='100%'))

In [17]:
import ollama
print(dir(ollama))

['AsyncClient', 'ChatResponse', 'Client', 'GenerateResponse', 'Message', 'Options', 'ProgressResponse', 'RequestError', 'ResponseError', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_client', '_types', 'chat', 'copy', 'create', 'delete', 'embed', 'embeddings', 'generate', 'list', 'ps', 'pull', 'push', 'show']


In [22]:
!ollama pull mxbai-embed-large

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   1% ▕                ▏ 4.0 MB/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   1% ▕                ▏ 8.5 MB/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   2% ▕                ▏  13 MB/669 M

In [31]:
from ollama import embeddings
from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

In [25]:
MyEmbeddings = embeddings(
    model="mxbai-embed-large",
)

In [32]:
MyEmbeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

C:\Users\franc\AppData\Local\Temp\ipykernel_1268\1079728633.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  MyEmbeddings = OllamaEmbeddings(


In [28]:
print(dir(embeddings))

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [33]:
vector_index = Neo4jVector.from_existing_graph(
    MyEmbeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [35]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)




In [42]:
!pip install openai



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
!pip install -qU langchain-openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
!pip install -qU langchain-ollama


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [71]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [68]:
# doesn't work with invoke
from langchain_ollama import ChatOllama
llm = ChatOllama( temperature=0, model="llama3.1",)


In [72]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [73]:
entity_chain.invoke("Who are Nonna Lucia and Giovanni Caruso?")

Entities(names=['Nonna Lucia', 'Giovanni Caruso'])

In [58]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [59]:
print(graph_retriever("Who is Nonna Lucia?"))

Nonna Lucia - IS_CULINARY_SAGE -> null
Nonna Lucia - IS_MENTOR_TO -> The Matriarch and Mentor
Nonna - TEACHES -> grandchildren
Sicilian cooking - IS_TEUGHT_BY -> Nonna
Lucia - IS_REFERRED_AS -> Nonna
Amico - LEARNED_FROM -> Nonna


In [99]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [100]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [101]:
chain.invoke(input="Who is Nonna Lucia? Did she teach anyone about restaurants or cooking?")

"Nonna Lucia is the matriarch of the Caruso family, a culinary sage who taught her grandchildren, including Amico, the art of Sicilian cooking. She's also the owner of Bella Vita."